# **데이터 불러오기**

In [14]:
import pandas as pd
import numpy as np

train=pd.read_csv('train_features.csv')
train_labels=pd.read_csv('train_labels.csv')
test=pd.read_csv('test_features.csv')
submission=pd.read_csv('sample_submission.csv')

In [22]:
train = train[['id','time','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']]
train = train.assign(loc = lambda x: np.hypot(np.hypot(x['gy_x'],x['gy_y']),x['gy_z']))
test = test[['id','time','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z']]
test = test.assign(loc = lambda x: np.hypot(np.hypot(x['gy_x'],x['gy_y']),x['gy_z']))
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,loc
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,44.010999
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,46.403958
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,50.851423
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,60.281107
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,64.446693
...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,132.293402
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,130.931134
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,129.739912
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,129.726288


In [15]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [23]:
X=tf.reshape(np.array(train.iloc[:,2:]),[-1, 600,7])
X.shape
X

<tf.Tensor: shape=(3125, 600, 7), dtype=float64, numpy=
array([[[ 1.20608659e+00, -1.79371436e-01, -1.48446664e-01, ...,
         -3.05490095e+01, -3.16761119e+01,  4.40109991e+01],
        [ 1.28769648e+00, -1.98973718e-01, -1.82443746e-01, ...,
         -3.91391031e+01, -2.49272160e+01,  4.64039584e+01],
        [ 1.30460912e+00, -1.95114490e-01, -2.53382483e-01, ...,
         -4.41225645e+01, -2.50196286e+01,  5.08514229e+01],
        ...,
        [ 1.02564277e+00, -2.27845498e-01, -3.54516277e-01, ...,
          2.57686541e+01,  1.89320704e+01,  3.20608096e+01],
        [ 1.03155302e+00, -3.87862493e-01, -2.77856696e-01, ...,
          2.86976939e+01,  2.06315770e+01,  3.66540338e+01],
        [ 1.13815923e+00, -4.26846194e-01, -4.30263172e-01, ...,
          2.16759504e+01,  3.21230070e+01,  4.18838726e+01]],

       [[-2.11794694e-01, -7.87595918e-02,  8.54626552e-01, ...,
          1.02111637e+01,  7.39034752e+00,  2.21650364e+01],
        [-2.56963191e-01, -1.85552045e-02,  8.0

In [5]:
y = tf.keras.utils.to_categorical(train_labels['label']) 
y.shape

(3125, 61)

In [6]:
#가벼운 모델 생성
model = Sequential()
model.add(LSTM(32, input_shape=(600,7)))
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(X,y, epochs=30, batch_size=128, validation_split=0.2)

Epoch 1/30
20/20 [==============================] - 7s 115ms/step - loss: 3.9613 - accuracy: 0.1742 - val_loss: 3.1191 - val_accuracy: 0.4976
Epoch 2/30
20/20 [==============================] - 1s 30ms/step - loss: 2.9419 - accuracy: 0.4836 - val_loss: 2.6865 - val_accuracy: 0.4976
Epoch 3/30
20/20 [==============================] - 1s 30ms/step - loss: 2.7236 - accuracy: 0.4792 - val_loss: 2.6412 - val_accuracy: 0.4976
Epoch 4/30
20/20 [==============================] - 1s 30ms/step - loss: 2.7018 - accuracy: 0.4655 - val_loss: 2.6030 - val_accuracy: 0.4976
Epoch 5/30
20/20 [==============================] - 1s 30ms/step - loss: 2.5784 - accuracy: 0.4889 - val_loss: 2.5635 - val_accuracy: 0.4976
Epoch 6/30
20/20 [==============================] - 1s 31ms/step - loss: 2.4881 - accuracy: 0.4903 - val_loss: 2.5313 - val_accuracy: 0.4976
Epoch 7/30
20/20 [==============================] - 1s 30ms/step - loss: 2.5105 - accuracy: 0.4747 - val_loss: 2.4933 - val_accuracy: 0.4992
Epoch 8/30
2

In [9]:
test_X=tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, 7])
test_X.shape

TensorShape([782, 600, 6])

In [10]:
prediction=model.predict(test_X)

ValueError: in user code:

    C:\Users\buleb\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\buleb\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\buleb\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\buleb\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\buleb\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re

In [26]:
prediction.shape

(782, 61)

In [27]:
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
submission.iloc[:,1:]=prediction

In [29]:
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000301,0.003290,0.000369,0.000338,0.015384,0.000580,0.006409,0.001503,0.000048,...,0.000179,0.000205,0.000167,0.001421,0.000473,0.000357,0.003223,0.000174,0.000795,0.000443
1,3126,0.000525,0.010528,0.000543,0.001359,0.003105,0.002556,0.005016,0.022472,0.017300,...,0.001372,0.000502,0.000550,0.009339,0.000749,0.001575,0.010912,0.002653,0.005687,0.004452
2,3127,0.002067,0.001232,0.000131,0.000051,0.029531,0.003229,0.003287,0.002732,0.000487,...,0.003329,0.000501,0.000281,0.000564,0.000389,0.002273,0.002984,0.004974,0.001399,0.000795
3,3128,0.000195,0.007442,0.000289,0.000281,0.003067,0.000305,0.001921,0.017881,0.011907,...,0.001529,0.000721,0.000104,0.000622,0.000217,0.000121,0.004863,0.002204,0.000537,0.010949
4,3129,0.000552,0.001173,0.000179,0.000093,0.008247,0.000080,0.000267,0.000792,0.000029,...,0.002180,0.000562,0.000029,0.000017,0.000385,0.000069,0.000395,0.004436,0.000084,0.000536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.001380,0.000483,0.000283,0.000670,0.003960,0.036610,0.058447,0.000376,0.000524,...,0.000610,0.001710,0.003780,0.036920,0.006222,0.007019,0.028875,0.000526,0.009594,0.000896
778,3903,0.005960,0.004659,0.000341,0.000159,0.042721,0.003638,0.004792,0.026260,0.000690,...,0.004596,0.001197,0.000886,0.000221,0.001182,0.004063,0.002688,0.008492,0.009667,0.006438
779,3904,0.000203,0.000204,0.000010,0.000003,0.004813,0.004014,0.002775,0.000650,0.000070,...,0.000422,0.000079,0.000038,0.002174,0.000058,0.002262,0.003412,0.000320,0.000333,0.000139
780,3905,0.002598,0.017268,0.037058,0.037534,0.007102,0.020979,0.012106,0.000882,0.000386,...,0.004275,0.000102,0.009638,0.104479,0.051520,0.018730,0.009739,0.003638,0.029044,0.000852


In [24]:
submission.to_csv('submission.csv', index=False)